In [1]:
# pip install geopy

import pandas as pd
from geopy.geocoders import Nominatim

In [2]:
unpreprocessed = pd.read_csv('신호위반_민원_단속카메라_위도경도_지역구분.csv', encoding='cp949')

In [3]:
unpreprocessed

,위도,경도,구분,시도,시군구
0,37.465612,127.974247,신호위반 단속카메라,강원특별자치도,횡성군
1,38.121015,128.208753,신호위반 단속카메라,강원특별자치도,인제군
2,38.148825,127.300836,신호위반 단속카메라,강원특별자치도,철원군
3,37.836160,128.844213,신호위반 단속카메라,강원특별자치도,강릉시
4,38.064880,128.175403,신호위반 단속카메라,강원특별자치도,인제군
...,...,...,...,...,...
79939,38.191803,128.579178,신호위반 민원,강원특별자치도,속초시
79940,35.126899,126.789615,신호위반 민원,광주,광산구
79941,37.259304,127.104538,신호위반 민원,경기도,용인시
79942,37.466927,126.896164,신호위반 민원,서울,금천구


위 데이터는 데이터 전처리가 되지 않은 상태(이상값, 결측치 존재)

In [4]:
unpreprocessed['시도'].unique()

array(['강원특별자치도', '서울', '경기도', '광주', '대전', '전라남도', '경상북도', '인천', '영등포동7가',
       '충청남도', '전라북도', '순천시', '여수시', '대구', '청도군', '칠곡군', '부산', '경상남도',
       '충청북도', '창원시', '통영시', '제주특별자치도', '서귀포시', '제주시', '파주시', '달성군',
       '수성구', '청주시', '신평군', '남양주시', '울산', '남구', '중구', '강서구', '부평구', '세종',
       '서구', '태안군', '보령시', '천안시', '광주시', '화성시', '군산시', '부안군', '구례군',
       '나주시', '화순군', '영암군', '장성군', '김천시', '포천시', '의정부시', '안산시', '아산시',
       '김포시', '영도구', '가평군', '성남시', '구리시', '평택시', '계양구', '청양군', '동구',
       '당진시', '무안군', '창녕군', '진주시', '중랑구', '동작구', '수원시', '고양시', '성북구',
       '신안군', '강남구', '담양군', '서초구', '경산시', '포항시'], dtype=object)

시도 컬럼에 영등포동7가, 순천시, 여수시, 청도군, 칠곡군, 창원시, 통영시, 서귀포시, 제주시, 파주시, 달성군, 수성구, 청주시, 신평군, 남양주시,
남구, 중구, 강서구, 부평구, 서구, 태안군, 보령시, 천안시, 광주시, 화성시, 군산시, 부안군, 구례군, 나주시, 화순군, 영암군, 장성군, 김천시,
포천시, 의정부시, 안산시, 아산시, 김포시, 영도구, 가평군, 성남시, 구리시, 평택시, 계양구, 청양군, 동구, 당진시, 무안군, 창녕군, 진주시,
중랑구, 동작구, 수원시, 고양시, 성북구, 신안군, 강남구, 담양군, 서초구, 경산시, 포항시가 포함되어 있음.

In [5]:
unpreprocessed['시군구'].unique()

array(['횡성군', '인제군', '철원군', '강릉시', '원주시', '홍천군', '성동구', '중랑구', '관악구',
       '서초구', '노원구', '성북구', '서울', '하남시', '중구', '종로구', '서대문구', '송파구',
       '금천구', '영등포구', '강동구', '은평구', '서구', '남구', '북구', '유성구', '화순군', '구로구',
       '삼척시', '부천시', '용산구', '도봉구', '강서구', '동작구', '광진구', '가평군', '남양주시',
       '구리시', '김천시', '이천시', '음성군', '마포구', '강남구', '강북구', '동대문구', '양주시',
       '의정부시', '포천시', '양천구', '성남시', '수원시', '안양시', '안산시', '광명시', '과천시',
       '평택시', '김포시', '여주시', '오산시', '화성시', '시흥시', '용인시', '남동구', '미추홀구',
       '부평구', '동구', '연수구', '강화군', '광주시', '의왕시', '군포시', '양평군', '강진군',
       '완도군', '목포시', '무안군', '장성군', '나주시', '영암군', '광산구', '해남군', '순천시',
       '여수시', '서산시', '장수군', '광양시', '학구리', '고흥군', '영광군', '진도군', '곡성군',
       '함평군', '구례군', '장흥군', '동두천시', '고양시', '연천군', '파주시', '춘천시', '문경시',
       '보성군', '남산동', '경주시', '경산시', '성주군', '울진군', '포항시', '구미시', '칠곡군',
       '안동시', '영천시', '고령군', '의성군', '영덕군', '담양군', '군위군', '상주시', '영주시',
       '예천군', '영양군', '금천면', '청송군', '청도군', '봉화군', '석적읍', '금정구', '사상구',
       '함양군'

'서울', '남산동', '석적읍' 등 이상치 존재

In [6]:
# 역지오코딩 하는 함수

def geocoding_reverse(lat_lng_str): 
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    address = geolocoder.reverse(lat_lng_str)

    return address

In [7]:
# 함수 사용 예
addr_list = []
address = geocoding_reverse('37.465612, 127.974247')

addr_list = str(address).split(', ')

print(addr_list)

['횡성로', '묵계리', '횡성읍', '횡성군', '강원특별자치도', '25241', '대한민국']


-1 인덱스 : 국가  
-2 인덱스 : 우편번호  
-3 인덱스 : 시도  
-4 인덱스 : 시군구  
(근데 항상 이 형식으로 리턴하지 않음)

In [8]:
addr_list = []
address = geocoding_reverse('34.534616, 128.385622')

addr_list = str(address).split(', ')

print(addr_list)

['통영시', '경상남도', '대한민국']


이런식으로 나오기도 한다. 그래서 데이터 저장 과정에서 이상치 발생

In [9]:
# 이상치데이터 예시 1
unpreprocessed[unpreprocessed['시도']=='영등포동7가']

,위도,경도,구분,시도,시군구
639,37.522935,126.904977,신호위반 단속카메라,영등포동7가,영등포구
16201,37.525579,126.911050,신호위반 민원,영등포동7가,영등포구
23976,37.525064,126.909658,신호위반 민원,영등포동7가,영등포구
28934,37.525168,126.909665,신호위반 민원,영등포동7가,영등포구
29074,37.525069,126.909617,신호위반 민원,영등포동7가,영등포구
30668,37.524982,126.909468,신호위반 민원,영등포동7가,영등포구
32654,37.524398,126.907808,신호위반 민원,영등포동7가,영등포구
39901,37.522896,126.905202,신호위반 민원,영등포동7가,영등포구
43710,37.522909,126.903842,신호위반 민원,영등포동7가,영등포구
44920,37.525369,126.911028,신호위반 민원,영등포동7가,영등포구


시도가 '영등포동7가'로 되어있는 데이터를 조회결과  
시군구는 '영등포구'로 다 들어가 있기 때문에  
'영등포동7가'를 서울로 바꿔야 한다.

In [10]:
addr_list = []
address = geocoding_reverse('37.522935, 126.904977')

addr_list = str(address).split(', ')

print(addr_list)

['75', '영중로', '영등포동', '영등포구', '영등포동7가', '07245', '대한민국']


In [11]:
index = unpreprocessed[unpreprocessed['시도']=='영등포동7가'].index

In [12]:
for i in index:
    unpreprocessed['시도'][i] = '서울'

/var/folders/lm/0qgrycpj4y52760rt_gw40q00000gn/T/ipykernel_7240/3355148287.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '서울'


In [13]:
unpreprocessed.iloc[[639, 16201]]

,위도,경도,구분,시도,시군구
639,37.522935,126.904977,신호위반 단속카메라,서울,영등포구
16201,37.525579,126.911050,신호위반 민원,서울,영등포구


'영등포동7가'였던 데이터가 '서울'로 변경되었다.  
이런식으로 수정 필요하고 NULL값도 찾아서 수정해야한다.

In [24]:
unpreprocessed = unpreprocessed.iloc[60000: ]

In [26]:
unpreprocessed

,위도,경도,구분,시도,시군구
60000,37.303383,126.847034,신호위반 민원,경기도,안산시
60001,35.113312,129.038866,신호위반 민원,부산,동구
60002,35.178600,126.929212,신호위반 민원,광주,북구
60003,37.604431,127.133162,신호위반 민원,경기도,구리시
60004,37.466625,126.891544,신호위반 민원,서울,금천구
...,...,...,...,...,...
79939,38.191803,128.579178,신호위반 민원,강원특별자치도,속초시
79940,35.126899,126.789615,신호위반 민원,광주,광산구
79941,37.259304,127.104538,신호위반 민원,경기도,용인시
79942,37.466927,126.896164,신호위반 민원,서울,금천구


In [27]:
unpreprocessed['시도'].unique()

array(['경기도', '부산', '광주', '서울', '인천', '경상남도', '전라남도', '대전', '경상북도', '울산',
       '제주특별자치도', '충청북도', '충청남도', '대구', '전라북도', '세종', '군산시', '강원특별자치도',
       '강남구', '영등포동7가', '강서구', '제주시', '서구', '당진시', '태안군', '구리시', '화순군',
       '수원시', '무안군', '김포시', '남구', '담양군', '안산시', '남양주시', '순천시', '천안시',
       '청주시', '서초구', '경산시', '여수시', '화성시', '포항시'], dtype=object)

In [28]:
unpreprocessed[unpreprocessed['시도']=='군산시']

,위도,경도,구분,시도,시군구
60138,35.947872,126.583007,신호위반 민원,군산시,산단로
60914,35.960558,126.561695,신호위반 민원,군산시,오식도동
62855,35.961943,126.586375,신호위반 민원,군산시,오식도동
72479,35.946289,126.561295,신호위반 민원,군산시,가도안1길
73224,35.947549,126.583452,신호위반 민원,군산시,새만금북로
73462,35.946946,126.575879,신호위반 민원,군산시,새만금북로
74001,35.960321,126.561086,신호위반 민원,군산시,오식도동
75195,35.973837,126.585681,신호위반 민원,군산시,동장산로


In [29]:
addr_list = []
address = geocoding_reverse('35.947872, 126.583007')

addr_list = str(address).split(', ')

print(addr_list)

['산단로', '군산시', '전라북도', '대한민국']


In [30]:
index = unpreprocessed[unpreprocessed['시도']=='군산시'].index

In [31]:
for i in index:
    unpreprocessed['시도'][i] = '전라북도'
    unpreprocessed['시군구'][i] = '군산시'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3989557645.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '전라북도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3989557645.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '군산시'


In [33]:
unpreprocessed[unpreprocessed['시군구']=='군산시']

,위도,경도,구분,시도,시군구
60084,35.956662,126.703574,신호위반 민원,전라북도,군산시
60138,35.947872,126.583007,신호위반 민원,전라북도,군산시
60338,35.961571,126.621179,신호위반 민원,전라북도,군산시
60339,35.961684,126.621135,신호위반 민원,전라북도,군산시
60342,35.975666,126.741300,신호위반 민원,전라북도,군산시
...,...,...,...,...,...
78290,35.965837,126.622229,신호위반 민원,전라북도,군산시
78412,35.968532,126.705764,신호위반 민원,전라북도,군산시
78570,35.963099,126.719730,신호위반 민원,전라북도,군산시
78578,35.969073,126.715093,신호위반 민원,전라북도,군산시


In [35]:
unpreprocessed.loc[[60138, 62855]]

,위도,경도,구분,시도,시군구
60138,35.947872,126.583007,신호위반 민원,전라북도,군산시
62855,35.961943,126.586375,신호위반 민원,전라북도,군산시


In [36]:
unpreprocessed[unpreprocessed['시도']=='강남구']

,위도,경도,구분,시도,시군구
60517,37.502347,127.04297,신호위반 민원,강남구,역삼2동


In [37]:
addr_list = []
address = geocoding_reverse('37.502347, 127.04297')

addr_list = str(address).split(', ')

print(addr_list)

['역삼동', '역삼2동', '강남구', '서울', '대한민국']


In [38]:
index = unpreprocessed[unpreprocessed['시도']=='강남구'].index
for i in index:
    unpreprocessed['시도'][i] = '서울'
    unpreprocessed['시군구'][i] = '강남구'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3512258706.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '서울'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3512258706.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '강남구'


In [39]:
unpreprocessed.loc[[60517]]

,위도,경도,구분,시도,시군구
60517,37.502347,127.04297,신호위반 민원,서울,강남구


In [40]:
unpreprocessed[unpreprocessed['시도']=='영등포동7가']

,위도,경도,구분,시도,시군구
60706,37.525437,126.906115,신호위반 민원,영등포동7가,영등포구
63938,37.525605,126.910872,신호위반 민원,영등포동7가,영등포구
68622,37.522795,126.903831,신호위반 민원,영등포동7가,영등포구
68789,37.523481,126.906963,신호위반 민원,영등포동7가,영등포구
69418,37.522559,126.906594,신호위반 민원,영등포동7가,영등포구
69420,37.522982,126.905281,신호위반 민원,영등포동7가,영등포구
69834,37.522909,126.905259,신호위반 민원,영등포동7가,영등포구
71978,37.524223,126.908203,신호위반 민원,영등포동7가,영등포구
74570,37.524854,126.909466,신호위반 민원,영등포동7가,영등포구
74891,37.525487,126.905515,신호위반 민원,영등포동7가,영등포구


In [41]:
addr_list = []
address = geocoding_reverse('37.525437, 126.906115')

addr_list = str(address).split(', ')

print(addr_list)

['신월여의지하도로', '영등포동', '영등포구', '영등포동7가', '07992', '대한민국']


In [43]:
index = unpreprocessed[unpreprocessed['시도']=='영등포동7가'].index
for i in index:
    unpreprocessed['시도'][i] = '서울'
    unpreprocessed['시군구'][i] = '영등포구'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2914556451.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '서울'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2914556451.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '영등포구'


In [44]:
unpreprocessed[unpreprocessed['시도']=='강서구']

,위도,경도,구분,시도,시군구
61646,35.097014,128.907561,신호위반 민원,강서구,명지동


In [45]:
addr_list = []
address = geocoding_reverse('35.097014, 128.907561')

addr_list = str(address).split(', ')

print(addr_list)

['명지동', '강서구', '부산', '대한민국']


In [46]:
index = unpreprocessed[unpreprocessed['시도']=='강서구'].index
for i in index:
    unpreprocessed['시도'][i] = '부산'
    unpreprocessed['시군구'][i] = '강서구'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\607471349.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '부산'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\607471349.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '강서구'


In [47]:
unpreprocessed[unpreprocessed['시도']=='제주시']

,위도,경도,구분,시도,시군구
61911,33.433215,126.743688,신호위반 민원,제주시,번영로
70654,33.416531,126.678208,신호위반 민원,제주시,수망리


In [48]:
addr_list = []
address = geocoding_reverse('33.433215,126.743688')

addr_list = str(address).split(', ')

print(addr_list)

['번영로', '제주시', '제주특별자치도', '대한민국']


In [49]:
index = unpreprocessed[unpreprocessed['시도']=='제주시'].index
for i in index:
    unpreprocessed['시도'][i] = '제주특별자치도'
    unpreprocessed['시군구'][i] = '제주시'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2926954710.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '제주특별자치도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2926954710.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '제주시'


In [50]:
unpreprocessed[unpreprocessed['시도']=='서구']

,위도,경도,구분,시도,시군구
62833,37.532753,126.640603,신호위반 민원,서구,청라2동
79579,37.596176,126.635875,신호위반 민원,서구,오류왕길동


In [51]:
addr_list = []
address = geocoding_reverse('37.532753, 126.640603')

addr_list = str(address).split(', ')

print(addr_list)

['청라2동', '서구', '인천', '대한민국']


In [52]:
addr_list = []
address = geocoding_reverse('37.596176, 126.635875')

addr_list = str(address).split(', ')

print(addr_list)

['단봉로139번길', '오류동', '오류왕길동', '서구', '인천', '대한민국']


In [53]:
index = unpreprocessed[unpreprocessed['시도']=='서구'].index
for i in index:
    unpreprocessed['시도'][i] = '인천'
    unpreprocessed['시군구'][i] = '서구'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\683561482.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '인천'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\683561482.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '서구'


In [54]:
unpreprocessed[unpreprocessed['시도']=='당진시']

,위도,경도,구분,시도,시군구
62909,37.025749,126.558854,신호위반 민원,당진시,장고항리
62911,37.027457,126.560316,신호위반 민원,당진시,장고항리


In [55]:
addr_list = []
address = geocoding_reverse('37.025749, 126.558854')

addr_list = str(address).split(', ')

print(addr_list)

['장고항로', '장고항리', '당진시', '충청남도', '대한민국']


In [56]:
index = unpreprocessed[unpreprocessed['시도']=='당진시'].index
for i in index:
    unpreprocessed['시도'][i] = '충청남도'
    unpreprocessed['시군구'][i] = '당진시'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\827960517.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '충청남도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\827960517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '당진시'


In [57]:
unpreprocessed[unpreprocessed['시도']=='태안군']

,위도,경도,구분,시도,시군구
64574,36.737930,126.133175,신호위반 민원,태안군,파도리
64575,36.737926,126.133187,신호위반 민원,태안군,파도리


In [59]:
addr_list = []
address = geocoding_reverse('36.737930, 126.133175')

addr_list = str(address).split(', ')

print(addr_list)

['파도길', '파도리', '태안군', '충청남도', '대한민국']


In [60]:
index = unpreprocessed[unpreprocessed['시도']=='태안군'].index
for i in index:
    unpreprocessed['시도'][i] = '충청남도'
    unpreprocessed['시군구'][i] = '태안군'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2694761328.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '충청남도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2694761328.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '태안군'


In [62]:
unpreprocessed[unpreprocessed['시도']=='구리시']

,위도,경도,구분,시도,시군구
65111,37.619168,127.143003,신호위반 민원,구리시,왕숙길벚교 쉼터
69858,37.619213,127.142992,신호위반 민원,구리시,왕숙길벚교 쉼터


In [63]:
addr_list = []
address = geocoding_reverse('37.619168,127.143003')

addr_list = str(address).split(', ')

print(addr_list)

['왕숙천로', '구리시', '경기도', '11908', '대한민국']


In [64]:
index = unpreprocessed[unpreprocessed['시도']=='구리시'].index
for i in index:
    unpreprocessed['시도'][i] = '경기도'
    unpreprocessed['시군구'][i] = '구리시'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\704732980.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '경기도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\704732980.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '구리시'


In [65]:
unpreprocessed[unpreprocessed['시도']=='화순군']

,위도,경도,구분,시도,시군구
65760,35.051724,126.958095,신호위반 민원,화순군,화순읍


In [66]:
addr_list = []
address = geocoding_reverse('35.051724, 126.958095')

addr_list = str(address).split(', ')

print(addr_list)

['오성로', '화순읍', '화순군', '전라남도', '대한민국']


In [67]:
index = unpreprocessed[unpreprocessed['시도']=='화순군'].index
for i in index:
    unpreprocessed['시도'][i] = '전라남도'
    unpreprocessed['시군구'][i] = '화순군'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\4025741389.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '전라남도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\4025741389.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '화순군'


In [68]:
unpreprocessed[unpreprocessed['시도']=='수원시']

,위도,경도,구분,시도,시군구
66134,37.265615,126.998782,신호위반 민원,수원시,권선구


In [69]:
addr_list = []
address = geocoding_reverse('37.265615, 126.998782')

addr_list = str(address).split(', ')

print(addr_list)

['권선구', '수원시', '경기도', '대한민국']


In [70]:
index = unpreprocessed[unpreprocessed['시도']=='수원시'].index
for i in index:
    unpreprocessed['시도'][i] = '경기도'
    unpreprocessed['시군구'][i] = '수원시'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2700206414.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '경기도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2700206414.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '수원시'


In [71]:
unpreprocessed[unpreprocessed['시도']=='무안군']

,위도,경도,구분,시도,시군구
66333,34.840377,126.500008,신호위반 민원,무안군,일로읍


In [72]:
addr_list = []
address = geocoding_reverse('34.840377, 126.500008')

addr_list = str(address).split(', ')

print(addr_list)

['일로읍', '무안군', '전라남도', '대한민국']


In [73]:
index = unpreprocessed[unpreprocessed['시도']=='무안군'].index
for i in index:
    unpreprocessed['시도'][i] = '전라남도'
    unpreprocessed['시군구'][i] = '무안군'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\562627888.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '전라남도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\562627888.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '무안군'


In [74]:
unpreprocessed[unpreprocessed['시도']=='김포시']

,위도,경도,구분,시도,시군구
66461,37.645263,126.627463,신호위반 민원,김포시,양곡리
69232,37.591223,126.792352,신호위반 민원,김포시,고촌읍


In [75]:
addr_list = []
address = geocoding_reverse('37.645263, 126.627463')

addr_list = str(address).split(', ')

print(addr_list)

['구래동', '양곡리', '김포시', '경기도', '대한민국']


In [76]:
index = unpreprocessed[unpreprocessed['시도']=='김포시'].index
for i in index:
    unpreprocessed['시도'][i] = '경기도'
    unpreprocessed['시군구'][i] = '김포시'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3532842967.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '경기도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3532842967.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '김포시'


In [77]:
unpreprocessed[unpreprocessed['시도']=='남구']

,위도,경도,구분,시도,시군구
67185,35.088543,126.889128,신호위반 민원,남구,NaN
73891,35.550393,129.263788,신호위반 민원,남구,신복로터리
75025,35.467382,129.360266,신호위반 민원,남구,선암동
77134,35.550060,129.264151,신호위반 민원,남구,신복로터리


In [78]:
addr_list = []
address = geocoding_reverse('35.088543, 126.889128')

addr_list = str(address).split(', ')

print(addr_list)

['남구', '광주', '대한민국']


In [79]:
addr_list = []
address = geocoding_reverse('35.550393,129.263788')

addr_list = str(address).split(', ')

print(addr_list)

['신복로터리', '남구', '울산', '대한민국']


In [80]:
addr_list = []
address = geocoding_reverse('35.467382,129.360266')

addr_list = str(address).split(', ')

print(addr_list)

['용연로', '황성동', '선암동', '남구', '울산', '대한민국']


In [81]:
unpreprocessed['시도'][67185] = '광주'
unpreprocessed['시군구'][67185] = '남구'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3853707987.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][67185] = '광주'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3853707987.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][67185] = '남구'


In [82]:
unpreprocessed[unpreprocessed['시도']=='남구']

,위도,경도,구분,시도,시군구
73891,35.550393,129.263788,신호위반 민원,남구,신복로터리
75025,35.467382,129.360266,신호위반 민원,남구,선암동
77134,35.550060,129.264151,신호위반 민원,남구,신복로터리


In [83]:
unpreprocessed.loc[[67185]]

,위도,경도,구분,시도,시군구
67185,35.088543,126.889128,신호위반 민원,광주,남구


In [84]:
index = unpreprocessed[unpreprocessed['시도']=='남구'].index
for i in index:
    unpreprocessed['시도'][i] = '울산'
    unpreprocessed['시군구'][i] = '남구'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\1244904936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '울산'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\1244904936.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '남구'


In [86]:
unpreprocessed[unpreprocessed['시도']=='담양군']

,위도,경도,구분,시도,시군구
68209,35.332859,126.98976,신호위반 민원,담양군,향교리


In [87]:
addr_list = []
address = geocoding_reverse('35.332859, 126.98976')

addr_list = str(address).split(', ')

print(addr_list)

['향교리', '담양군', '전라남도', '대한민국']


In [88]:
index = unpreprocessed[unpreprocessed['시도']=='담양군'].index
for i in index:
    unpreprocessed['시도'][i] = '전라남도'
    unpreprocessed['시군구'][i] = '담양군'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2361142269.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '전라남도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2361142269.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '담양군'


In [89]:
unpreprocessed[unpreprocessed['시도']=='안산시']

,위도,경도,구분,시도,시군구
68454,37.327819,126.843831,신호위반 민원,안산시,상록구
78817,37.328026,126.843627,신호위반 민원,안산시,상록구


In [90]:
addr_list = []
address = geocoding_reverse('37.327819, 126.843831')

addr_list = str(address).split(', ')

print(addr_list)

['예술광장1로', '월피동', '상록구', '안산시', '경기도', '대한민국']


In [91]:
index = unpreprocessed[unpreprocessed['시도']=='안산시'].index
for i in index:
    unpreprocessed['시도'][i] = '경기도'
    unpreprocessed['시군구'][i] = '안산시'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3978736549.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '경기도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3978736549.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '안산시'


In [93]:
unpreprocessed[unpreprocessed['시도']=='남양주시']

,위도,경도,구분,시도,시군구
68906,37.733852,127.294444,신호위반 민원,남양주시,수동면
75629,37.708957,127.322391,신호위반 민원,남양주시,수동면


In [94]:
addr_list = []
address = geocoding_reverse('37.708957, 127.322391')

addr_list = str(address).split(', ')

print(addr_list)

['수동면', '남양주시', '경기도', '대한민국']


In [95]:
index = unpreprocessed[unpreprocessed['시도']=='남양주시'].index
for i in index:
    unpreprocessed['시도'][i] = '경기도'
    unpreprocessed['시군구'][i] = '남양주시'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\1826305420.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '경기도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\1826305420.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '남양주시'


In [96]:
unpreprocessed[unpreprocessed['시도']=='순천시']

,위도,경도,구분,시도,시군구
71724,34.952876,127.531783,신호위반 민원,순천시,상삼리


In [97]:
addr_list = []
address = geocoding_reverse('34.952876, 127.531783')

addr_list = str(address).split(', ')

print(addr_list)

['조례보도육교', '상삼리', '순천시', '전라남도', '대한민국']


In [98]:
index = unpreprocessed[unpreprocessed['시도']=='순천시'].index
for i in index:
    unpreprocessed['시도'][i] = '전라남도'
    unpreprocessed['시군구'][i] = '순천시'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3371976130.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '전라남도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3371976130.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '순천시'


In [99]:
unpreprocessed[unpreprocessed['시도']=='천안시']

,위도,경도,구분,시도,시군구
74207,36.810344,127.105455,신호위반 민원,천안시,서북구
75663,36.801888,127.112480,신호위반 민원,천안시,서북구


In [100]:
addr_list = []
address = geocoding_reverse('36.801888,127.112480')

addr_list = str(address).split(', ')

print(addr_list)

['불당동원형육교', '불당동', '서북구', '천안시', '충청남도', '대한민국']


In [101]:
index = unpreprocessed[unpreprocessed['시도']=='천안시'].index
for i in index:
    unpreprocessed['시도'][i] = '충청남도'
    unpreprocessed['시군구'][i] = '천안시'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3175769997.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '충청남도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3175769997.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '천안시'


In [102]:
unpreprocessed[unpreprocessed['시도']=='청주시']

,위도,경도,구분,시도,시군구
74751,36.732063,127.377393,신호위반 민원,청주시,청원구


In [103]:
addr_list = []
address = geocoding_reverse('36.732063, 127.377393')

addr_list = str(address).split(', ')

print(addr_list)

['청원구', '청주시', '충청북도', '대한민국']


In [104]:
index = unpreprocessed[unpreprocessed['시도']=='청주시'].index
for i in index:
    unpreprocessed['시도'][i] = '충청북도'
    unpreprocessed['시군구'][i] = '청주시'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\1833463981.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '충청북도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\1833463981.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '청주시'


In [105]:
unpreprocessed[unpreprocessed['시도']=='서초구']

,위도,경도,구분,시도,시군구
74904,37.45455,127.0603,신호위반 민원,서초구,안골


In [106]:
addr_list = []
address = geocoding_reverse('37.45455, 127.0603')

addr_list = str(address).split(', ')

print(addr_list)

['내곡동', '안골', '서초구', '서울', '대한민국']


In [107]:
index = unpreprocessed[unpreprocessed['시도']=='서초구'].index
for i in index:
    unpreprocessed['시도'][i] = '서울'
    unpreprocessed['시군구'][i] = '서초구'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2825295371.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '서울'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2825295371.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '서초구'


In [108]:
unpreprocessed[unpreprocessed['시도']=='경산시']

,위도,경도,구분,시도,시군구
74995,35.875124,128.820421,신호위반 민원,경산시,진량읍


In [109]:
addr_list = []
address = geocoding_reverse('35.875124,128.820421')

addr_list = str(address).split(', ')

print(addr_list)

['진량읍', '경산시', '경상북도', '대한민국']


In [110]:
index = unpreprocessed[unpreprocessed['시도']=='경산시'].index
for i in index:
    unpreprocessed['시도'][i] = '경상북도'
    unpreprocessed['시군구'][i] = '경산시'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\850455930.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '경상북도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\850455930.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '경산시'


In [111]:
unpreprocessed[unpreprocessed['시도']=='여수시']

,위도,경도,구분,시도,시군구
75904,34.63868,127.776947,신호위반 민원,여수시,죽포리


In [112]:
addr_list = []
address = geocoding_reverse('34.63868, 127.776947')

addr_list = str(address).split(', ')

print(addr_list)

['돌산로', '죽포리', '여수시', '전라남도', '대한민국']


In [113]:
index = unpreprocessed[unpreprocessed['시도']=='여수시'].index
for i in index:
    unpreprocessed['시도'][i] = '전라남도'
    unpreprocessed['시군구'][i] = '여수시'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\745179017.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '전라남도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\745179017.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '여수시'


In [114]:
unpreprocessed[unpreprocessed['시도']=='화성시']

,위도,경도,구분,시도,시군구
77732,37.033302,126.763595,신호위반 민원,화성시,매향리


In [115]:
addr_list = []
address = geocoding_reverse('37.033302, 126.763595')

addr_list = str(address).split(', ')

print(addr_list)

['기아자동차로', '매향리', '화성시', '경기도', '대한민국']


In [116]:
index = unpreprocessed[unpreprocessed['시도']=='화성시'].index
for i in index:
    unpreprocessed['시도'][i] = '경기도'
    unpreprocessed['시군구'][i] = '화성시'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\384184761.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '경기도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\384184761.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '화성시'


In [117]:
unpreprocessed[unpreprocessed['시도']=='포항시']

,위도,경도,구분,시도,시군구
79374,36.078597,129.396754,신호위반 민원,포항시,북구


In [118]:
addr_list = []
address = geocoding_reverse('36.078597, 129.396754')

addr_list = str(address).split(', ')

print(addr_list)

['장량동', '북구', '포항시', '경상북도', '대한민국']


In [119]:
index = unpreprocessed[unpreprocessed['시도']=='포항시'].index
for i in index:
    unpreprocessed['시도'][i] = '경상북도'
    unpreprocessed['시군구'][i] = '포항시'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2647504926.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '경상북도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2647504926.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '포항시'


In [121]:
unpreprocessed['시군구'].unique()

array(['안산시', '동구', '북구', '구리시', '금천구', '화성시', '부천시', '연수구', '창원시', '목포시',
       '사하구', '관악구', '유성구', '포항시', '평택시', '오산시', '수원시', '용인시', '울주군',
       '시흥시', '연제구', '안양시', '은평구', '광진구', '양주시', '서구', '제주시', '고양시',
       '성남시', '동작구', '나주시', '청주시', '보령시', '수성구', '남동구', '경주시', '여수시',
       '강남구', '의왕시', '순천시', '대덕구', '아산시', '부산', '광산구', '서귀포시', '김포시',
       '중구', '파주시', '안성시', '태안군', '달서구', '광양시', '군산시', '조치원읍', '해운대구',
       '담양군', '강서구', '경산시', '수영구', '영등포구', '계양구', '남양주시', '음성군', '서대문구',
       '미추홀구', '광명시', '송파구', '동래구', '천안시', '진주시', '용포리', '새롬동', '봉안리',
       '전주시', '서초구', '남구', '구미시', '성동구', '부산진구', '김해시', '의정부시', '도봉구',
       '강동구', '금정구', '사상구', '상주시', '구로구', '진천군', '서산시', '서울', '속초시',
       '안동시', '양양군', '제천시', '창녕군', '김제시', '동대문구', '성북구', '용산구', '양산시',
       '광주시', '이천시', '기장군', '옥천군', '하남시', '당진시', '의성군', '춘천시', '함양군',
       '마포구', '강릉시', '양평군', '고창군', '노원구', '동해시', '정읍시', '나성동', '영주시',
       '부평구', '영천시', '익산시', '순창군', '칠곡군', '밀양시', '대구', '장재리', '사천시',
       '충주시

In [122]:
unpreprocessed[unpreprocessed['시군구']=='부산']

,위도,경도,구분,시도,시군구
60067,35.087319,128.831410,신호위반 민원,경상남도,부산
62221,35.298256,129.082981,신호위반 민원,경상남도,부산
62355,35.213298,128.987440,신호위반 민원,경상남도,부산
63584,35.214529,128.988375,신호위반 민원,경상남도,부산
63814,35.293765,129.087340,신호위반 민원,경상남도,부산
64235,35.083177,128.824087,신호위반 민원,경상남도,부산
64382,35.100276,128.824495,신호위반 민원,경상남도,부산
64982,35.086063,128.824692,신호위반 민원,경상남도,부산
65218,35.214392,128.988313,신호위반 민원,경상남도,부산
65986,35.087235,128.831386,신호위반 민원,경상남도,부산


In [133]:
addr_list = []
address = geocoding_reverse('35.214230	128.988309')

addr_list = str(address).split(', ')

print(addr_list)

['공항로', '대저1동', '강서구', '부산', '경상남도', '46703', '대한민국']


In [134]:
index = unpreprocessed[unpreprocessed['시군구']=='부산'].index
for i in index:
    unpreprocessed['시도'][i] = '부산'
    unpreprocessed['시군구'][i] = '강서구'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\1707283076.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '부산'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\1707283076.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '강서구'


In [155]:
unpreprocessed[unpreprocessed['시군구']=='인천']

,위도,경도,구분,시도,시군구
62962,37.619474,126.695065,신호위반 민원,경기도,인천
64597,37.487708,126.740358,신호위반 민원,경기도,인천
65500,37.593770,126.717372,신호위반 민원,경기도,인천
65823,37.508530,126.734402,신호위반 민원,경기도,인천
65840,37.510429,126.743641,신호위반 민원,경기도,인천
67376,37.589558,126.718491,신호위반 민원,경기도,인천
68741,37.508562,126.736403,신호위반 민원,경기도,인천
73070,37.491186,126.740583,신호위반 민원,경기도,인천
74841,37.232913,126.537225,신호위반 민원,경기도,인천
75103,37.592327,126.713570,신호위반 민원,경기도,인천


In [173]:
addr_list = []
address = geocoding_reverse('37.508530	126.734402')

addr_list = str(address).split(', ')

print(addr_list)

['701동', '27', '체육관로', '삼산동', '부평구', '인천', '경기도', '14505', '대한민국']


In [172]:
unpreprocessed['시도'][75103] = '인천'
unpreprocessed['시군구'][75103] = '서구'
unpreprocessed['시도'][76537] = '인천'
unpreprocessed['시군구'][76537] = '서구'
unpreprocessed['시도'][77741] = '인천'
unpreprocessed['시군구'][77741] = '서구'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3051968596.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][75103] = '인천'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3051968596.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][75103] = '서구'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3051968596.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][76537] =

In [174]:
index = unpreprocessed[unpreprocessed['시군구']=='인천'].index
for i in index:
    unpreprocessed['시도'][i] = '인천'
    unpreprocessed['시군구'][i] = '부평구'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3719111121.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '인천'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3719111121.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '부평구'


In [180]:
unpreprocessed[unpreprocessed['시군구']=='금정동']

,위도,경도,구분,시도,시군구
65775,37.354262,126.932873,신호위반 민원,경기도,금정동
65776,37.354128,126.932975,신호위반 민원,경기도,금정동
76413,37.351982,126.930604,신호위반 민원,경기도,금정동


In [181]:
addr_list = []
address = geocoding_reverse('37.354262	126.932873')

addr_list = str(address).split(', ')

print(addr_list)

['군포산본 율곡아파트', '오금동', '금정동', '경기도', '15864', '대한민국']


In [182]:
index = unpreprocessed[unpreprocessed['시군구']=='금정동'].index
for i in index:
    unpreprocessed['시도'][i] = '경기도'
    unpreprocessed['시군구'][i] = '군포시'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2836909892.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '경기도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2836909892.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '군포시'


In [183]:
unpreprocessed[unpreprocessed['시군구']=='오전동']

,위도,경도,구분,시도,시군구
67474,37.350052,126.976143,신호위반 민원,경기도,오전동


In [184]:
addr_list = []
address = geocoding_reverse('37.350052	126.976143')

addr_list = str(address).split(', ')

print(addr_list)

['오전천로', '고천동', '오전동', '경기도', '16061', '대한민국']


In [185]:
index = unpreprocessed[unpreprocessed['시군구']=='오전동'].index
for i in index:
    unpreprocessed['시도'][i] = '경기도'
    unpreprocessed['시군구'][i] = '의왕시'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3482540114.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '경기도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3482540114.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '의왕시'


In [190]:
unpreprocessed[unpreprocessed['시군구']=='안중읍']

,위도,경도,구분,시도,시군구
61370,37.002782,126.959363,신호위반 민원,경기도,안중읍
67653,36.987596,126.900400,신호위반 민원,경기도,안중읍
78065,36.979764,126.938861,신호위반 민원,경기도,안중읍


In [191]:
addr_list = []
address = geocoding_reverse('37.002782	126.959363')

addr_list = str(address).split(', ')

print(addr_list)

['서동대로', '죽리', '안중읍', '경기도', '17818', '대한민국']


In [192]:
index = unpreprocessed[unpreprocessed['시군구']=='안중읍'].index
for i in index:
    unpreprocessed['시도'][i] = '경기도'
    unpreprocessed['시군구'][i] = '평택시'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\16460997.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '경기도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\16460997.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '평택시'


In [194]:
unpreprocessed[unpreprocessed['시군구']=='산본동']

,위도,경도,구분,시도,시군구
68262,37.358159,126.924192,신호위반 민원,경기도,산본동
69055,37.365852,126.928759,신호위반 민원,경기도,산본동
73158,37.357772,126.924150,신호위반 민원,경기도,산본동
75630,37.356987,126.931627,신호위반 민원,경기도,산본동
75844,37.363686,126.930610,신호위반 민원,경기도,산본동
76260,37.357590,126.931374,신호위반 민원,경기도,산본동


In [195]:
addr_list = []
address = geocoding_reverse('37.358159	126.924192	')

addr_list = str(address).split(', ')

print(addr_list)

['고산로', '궁내동', '산본동', '경기도', '15825', '대한민국']


In [196]:
index = unpreprocessed[unpreprocessed['시군구']=='산본동'].index
for i in index:
    unpreprocessed['시도'][i] = '경기도'
    unpreprocessed['시군구'][i] = '군포시'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3830606676.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '경기도'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3830606676.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '군포시'


In [198]:
unpreprocessed[unpreprocessed['시군구']=='광주']

,위도,경도,구분,시도,시군구
73217,35.078476,126.864563,신호위반 민원,전라남도,광주


In [199]:
addr_list = []
address = geocoding_reverse('35.078476	126.864563')

addr_list = str(address).split(', ')

print(addr_list)

['산남로', '남구', '광주', '전라남도', '58227', '대한민국']


In [200]:
index = unpreprocessed[unpreprocessed['시군구']=='광주'].index
for i in index:
    unpreprocessed['시도'][i] = '광주'
    unpreprocessed['시군구'][i] = '남구'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2518609864.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '광주'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2518609864.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = '남구'


In [202]:
unpreprocessed['시군구'].unique()

array(['안산시', '동구', '북구', '구리시', '금천구', '화성시', '부천시', '연수구', '창원시', '목포시',
       '사하구', '관악구', '유성구', '포항시', '평택시', '오산시', '수원시', '용인시', '울주군',
       '시흥시', '연제구', '안양시', '은평구', '광진구', '양주시', '서구', '제주시', '고양시',
       '성남시', '동작구', '나주시', '청주시', '보령시', '수성구', '남동구', '경주시', '여수시',
       '강남구', '의왕시', '순천시', '대덕구', '아산시', '강서구', '광산구', '서귀포시', '김포시',
       '중구', '파주시', '안성시', '태안군', '달서구', '광양시', '군산시', None, '해운대구',
       '담양군', '경산시', '수영구', '영등포구', '계양구', '남양주시', '음성군', '서대문구', '미추홀구',
       '광명시', '송파구', '동래구', '천안시', '진주시', '전주시', '서초구', '남구', '구미시',
       '성동구', '부산진구', '김해시', '의정부시', '도봉구', '강동구', '금정구', '사상구', '상주시',
       '구로구', '진천군', '서산시', '서울', '속초시', '안동시', '양양군', '제천시', '창녕군',
       '김제시', '동대문구', '성북구', '용산구', '양산시', '광주시', '이천시', '기장군', '옥천군',
       '하남시', '당진시', '의성군', '춘천시', '함양군', '마포구', '강릉시', '양평군', '고창군',
       '노원구', '동해시', '정읍시', '영주시', '부평구', '영천시', '익산시', '순창군', '칠곡군',
       '밀양시', '대구', '사천시', '충주시', '논산시', '양천구', '영암군', '화순군', '거제시',
       '곡성군'

In [201]:
index = unpreprocessed[unpreprocessed['시도']=='세종'].index
for i in index:
    unpreprocessed['시도'][i] = '세종'
    unpreprocessed['시군구'][i] = None

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2946284857.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][i] = '세종'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2946284857.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][i] = None


In [232]:
unpreprocessed[unpreprocessed['시군구']=='서울']

,위도,경도,구분,시도,시군구


In [222]:
unpreprocessed['시도'][60258] = '서울'
unpreprocessed['시군구'][60258] = '금천구'
unpreprocessed['시도'][60687] = '서울'
unpreprocessed['시군구'][60687] = '강서구'
unpreprocessed['시도'][63293] = '서울'
unpreprocessed['시군구'][63293] = '강동구'
unpreprocessed['시도'][63336] = '서울'
unpreprocessed['시군구'][63336] = '양천구'
unpreprocessed['시도'][64473] = '서울'
unpreprocessed['시군구'][64473] = '강서구'
unpreprocessed['시도'][65742] = '서울'
unpreprocessed['시군구'][65742] = '강서구'
unpreprocessed['시도'][65922] = '서울'
unpreprocessed['시군구'][65922] = '강서구'
unpreprocessed['시도'][67134] = '서울'
unpreprocessed['시군구'][67134] = '광진구'
unpreprocessed['시도'][67534] = '서울'
unpreprocessed['시군구'][67534] = '송파구'
unpreprocessed['시도'][67760] = '서울'
unpreprocessed['시군구'][67760] = '양천구'
unpreprocessed['시도'][67994] = '서울'
unpreprocessed['시군구'][67994] = '강서구'
unpreprocessed['시도'][68332] = '서울'
unpreprocessed['시군구'][68332] = '강서구'
unpreprocessed['시도'][68600] = '서울'
unpreprocessed['시군구'][68600] = '구로구'
unpreprocessed['시도'][69445] = '서울'
unpreprocessed['시군구'][69445] = '서초구'
unpreprocessed['시도'][69999] = '서울'
unpreprocessed['시군구'][69999] = '강남구'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3825244921.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][60258] = '서울'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3825244921.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][60258] = '금천구'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\3825244921.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][60687] 

In [231]:
unpreprocessed['시도'][70005] = '서울'
unpreprocessed['시군구'][70005] = '강남구'
unpreprocessed['시도'][73970] = '서울'
unpreprocessed['시군구'][73970] = '강동구'
unpreprocessed['시도'][74170] = '서울'
unpreprocessed['시군구'][74170] = '강남구'
unpreprocessed['시도'][74613] = '서울'
unpreprocessed['시군구'][74613] = '강서구'
unpreprocessed['시도'][76250] = '서울'
unpreprocessed['시군구'][76250] = '강동구'
unpreprocessed['시도'][76996] = '서울'
unpreprocessed['시군구'][76996] = '강서구'
unpreprocessed['시도'][77738] = '서울'
unpreprocessed['시군구'][77738] = '금천구'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2305595081.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][70005] = '서울'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2305595081.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][70005] = '강남구'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\2305595081.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][73970] 

In [230]:
addr_list = []
address = geocoding_reverse('37.464834	126.884171')

addr_list = str(address).split(', ')

print(addr_list)

['안양천로', '독산1동', '금천구', '서울', '경기도', '14306', '대한민국']


In [ ]:
index = unpreprocessed[unpreprocessed['시군구']=='광주'].index
for i in index:
    unpreprocessed['시도'][i] = '광주'
    unpreprocessed['시군구'][i] = '남구'

In [233]:
unpreprocessed[unpreprocessed['시군구']=='대구']

,위도,경도,구분,시도,시군구
60624,35.837415,128.724627,신호위반 민원,경상북도,대구
63328,35.836472,128.717549,신호위반 민원,경상북도,대구
64474,35.876397,128.757630,신호위반 민원,경상북도,대구
68595,35.860098,128.716680,신호위반 민원,경상북도,대구


In [237]:
addr_list = []
address = geocoding_reverse('35.860098	128.716680')

addr_list = str(address).split(', ')

print(addr_list)

['경안로', '대평동', '고산3동', '수성구', '대구', '경상북도', '38624', '대한민국']


In [238]:
unpreprocessed['시도'][60624] = '대구'
unpreprocessed['시군구'][60624] = '수성구'
unpreprocessed['시도'][63328] = '대구'
unpreprocessed['시군구'][63328] = '수성구'
unpreprocessed['시도'][64474] = '대구'
unpreprocessed['시군구'][64474] = '동구'
unpreprocessed['시도'][68595] = '대구'
unpreprocessed['시군구'][68595] = '수성구'

C:\Users\82107\AppData\Local\Temp\ipykernel_11724\1329718530.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][60624] = '대구'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\1329718530.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시군구'][60624] = '수성구'
C:\Users\82107\AppData\Local\Temp\ipykernel_11724\1329718530.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpreprocessed['시도'][63328] 

In [240]:
unpreprocessed.to_csv('D.csv',  encoding='cp949', index=False)